In [258]:
import numpy as np
import pandas as pd
import astropy as ap
import astropy.time as atime
from astropy import units as u
import healpy as hp

from scipy.stats import norm
from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt

import yaml

In [ ]:
# pick random point in northern hemisphere
# pick random distance for event
# pick random radius in interval
# generate random (gaussian?) probabilities within this radius
# save probabilities as healpix map / fits file NESTED

# mean map area should stay same?
# 

In [280]:
def GenerateRandoms(long_range: list, lat_range: list, size: int, seed=1234):
    """Generade random value pairs for longitude, latitude, fwhm, trigger time, distance to be used in map generation.
    """

    np.random.seed(seed)

    longitudes = np.random.uniform(long_range[0], long_range[1], size=size)
    latitudes = np.random.uniform(lat_range[0], lat_range[1], size=size)

    fwhms = np.random.uniform(0.4, 1.2, size=size)

    distances = np.random.uniform(0, 2000, size=size)
    dist_uncs = np.random.normal(240, 80, size=size)

    trigger_times = np.random.uniform(1197102054.0, 1378023633.0, size=size)

    

    return pd.DataFrame({"longitude": longitudes, "latitude": latitudes, "fwhm": fwhms, "trigger_time": trigger_times, "distance": distances, "dist_unc": dist_uncs})


def GenerateMap(nside, latitude, longitude, fwhm):
    npix = hp.nside2npix(nside)

    pixels = np.zeros(npix)


    source_vector = hp.ang2vec(theta=longitude, phi=latitude, lonlat=True)
    #print(source_vector)


    radius = (3*u.deg).to_value(u.radian)

    disc_pix = hp.query_disc(nside, source_vector, radius=radius, nest=True)
    #print(disc_pix)

    pixels[disc_pix] = 100

    smoothed_pix = hp.smoothing(pixels, fwhm=fwhm, nest=True)
    smoothed_pix /= np.sum(smoothed_pix)

    return smoothed_pix

In [284]:
nside = 64
long_range = [0, 360]
lat_range = [0, 90]
size = 20

seed = 1234

map_save_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp"

jobfile_write_file = "../simulated_maps/simulated_events.yml"

#jobfile_dict = yaml.safe_load(open(jobfile_write_file))
jobfile_dict = None
if jobfile_dict == None:
    jobfile_dict = {}

random_coords = GenerateRandoms(long_range=long_range, lat_range=lat_range, size=size, seed=seed)

for index, row in random_coords.iterrows():
    map_pix = GenerateMap(nside=nside, latitude=row["latitude"], longitude=row["longitude"], fwhm=row["fwhm"])
    
    #hp.projview(map_pix, coord=["G"], graticule=True, graticule_labels=True, projection_type="mollweide",
        #xtick_label_color="w", longitude_grid_spacing=30, title="both contour", nest=True, rot=(0, 0))

    event_name = f"sim_{row['longitude']:.2f}_{row['latitude']:.2f}_{row['trigger_time']:.1f}"
    
    map_name = f"{map_save_dir}/{event_name}.fits.gz"

    print("Saving map: ", map_name)

    hdr = []
    hdr.append(("DISTMEAN", row["distance"]))
    hdr.append(("DISTSTD", row["dist_unc"]))

    trigger_time = atime.Time(row["trigger_time"], format="gps")

    

    trigger_time.format = "fits"

    hdr.append( ("DATE-OBS", trigger_time.value))
    hdr.append(("ORDERING", "NESTED"))

    tmp_dict = {"map_name": f"{event_name}.fits.gz", "trigger_jd": trigger_time.jd1}

    jobfile_dict[event_name] = tmp_dict

    hp.write_map(filename=map_name, nest=True, m=map_pix, extra_header=hdr, overwrite=True, dtype='float64')

with open(jobfile_write_file, "w") as file:
    yaml.dump(jobfile_dict, file)

Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_68.95_32.84_1335890005.7.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_223.96_55.39_1325215403.4.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_157.58_6.78_1341272523.1.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_282.73_33.19_1298013023.8.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_280.79_83.98_1371842724.4.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_98.13_58.62_1223725912.7.fits.gz
Saving map:  /mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-simulated/tmp/sim_99.53_35.75_1202465836.1.fits